In [1]:
import tensorflow as tf

train_data, valid_data = tf.keras.utils.image_dataset_from_directory(
    "./cats_vs_dogs",
    label_mode="categorical",
    image_size=(150, 150),
    validation_split=0.3,
    subset="both",
    seed=0,
)


Found 1877 files belonging to 2 classes.
Using 1314 files for training.
Using 563 files for validation.


In [2]:
classes = train_data.class_names
n_classes = len(classes)
classes

['cat', 'dog']

In [3]:
n_batches = valid_data.cardinality()
n_batches

<tf.Tensor: shape=(), dtype=int64, numpy=18>

In [4]:
test_size = n_batches // 2
test_data = valid_data.take(test_size)
valid_data = valid_data.skip(test_size)

In [5]:
for transform in [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
]:
    train_data = train_data.map(lambda x, y: (transform(x), y))

In [6]:
def create_model(hp=None):
    base_model = tf.keras.applications.Xception(
        weights="imagenet",
        input_shape=(150, 150, 3),
        include_top=False,
    )

    base_model.trainable = False

    inputs = tf.keras.Input(shape=(150, 150, 3))
    scaling = tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
    outputs = scaling(inputs)
    outputs = base_model(outputs, training=False)
    outputs = tf.keras.layers.GlobalAveragePooling2D()(outputs)

    hp_layers = hp.Int("layers", min_value=1, max_value=3, step=1) if hp else 1
    hp_units = hp.Int("units", min_value=32, max_value=128, step=32) if hp else 32
    
    for _ in range(hp_layers):
        outputs = tf.keras.layers.Dense(hp_units)(outputs)

    outputs = tf.keras.layers.Dense(n_classes)(outputs)
    model = tf.keras.Model(inputs, outputs)
    
    model.compile(
        optimizer=tf.keras.optimizers.AdamW(),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()],
    )

    return model

model = create_model()
model.summary(show_trainable=True)


Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling (Rescaling)       (None, 150, 150, 3)       0         Y          
                                                                            
 xception (Functional)       (None, 5, 5, 2048)        2086148   N          
                                                       0                    
                                                                            
 global_average_pooling2d (  (None, 2048)              0         Y          
 GlobalAveragePooling2D)                                                    
                                                                            
 dense (Dense)               (None, 32)                65568

In [7]:
# !pip install keras-tuner

import keras_tuner as kt
import numpy as np

tuner = kt.Hyperband(
    hypermodel=create_model,
    objective="categorical_accuracy",
    max_epochs=10,
    factor=3,
    project_name="cats_vs_dogs_hp"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

X_train = np.concatenate([x for x, _ in train_data], axis=0)
y_train = np.concatenate([y for _, y in train_data], axis=0)

tuner.search(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 18 Complete [00h 01m 38s]
categorical_accuracy: 0.6707897186279297

Best categorical_accuracy So Far: 0.70123690366745
Total elapsed time: 00h 14m 30s


In [24]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

val_acc_per_epoch = history.history["val_categorical_accuracy"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print("Best epoch: %d" % (best_epoch,))

Epoch 1/10
33/33 [==============================] - 12s 333ms/step - loss: 0.9944 - categorical_accuracy: 0.5214 - val_loss: 0.6946 - val_categorical_accuracy: 0.4905
Epoch 2/10
33/33 [==============================] - 13s 391ms/step - loss: 0.6882 - categorical_accuracy: 0.4643 - val_loss: 0.6953 - val_categorical_accuracy: 0.4905
Epoch 3/10
33/33 [==============================] - 15s 456ms/step - loss: 0.6844 - categorical_accuracy: 0.4386 - val_loss: 0.6982 - val_categorical_accuracy: 0.5057
Epoch 4/10
33/33 [==============================] - 15s 453ms/step - loss: 0.6774 - categorical_accuracy: 0.4291 - val_loss: 0.7045 - val_categorical_accuracy: 0.5019
Epoch 5/10
33/33 [==============================] - 15s 453ms/step - loss: 0.6912 - categorical_accuracy: 0.4415 - val_loss: 0.7151 - val_categorical_accuracy: 0.4829
Epoch 6/10
33/33 [==============================] - 15s 455ms/step - loss: 0.6683 - categorical_accuracy: 0.4110 - val_loss: 0.7422 - val_categorical_accuracy: 0.482

In [25]:
X_test = np.concatenate([x for x, _ in test_data], axis=0)
y_test = np.concatenate([y for _, y in test_data], axis=0)

hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(X_test, y_test, epochs=best_epoch, validation_split=0.2)

Epoch 1/3
8/8 [==============================] - 5s 495ms/step - loss: 1.2009 - categorical_accuracy: 0.5087 - val_loss: 0.5547 - val_categorical_accuracy: 0.6034
Epoch 2/3
8/8 [==============================] - 3s 431ms/step - loss: 0.4882 - categorical_accuracy: 0.7652 - val_loss: 0.4687 - val_categorical_accuracy: 0.8966
Epoch 3/3
8/8 [==============================] - 3s 440ms/step - loss: 0.4101 - categorical_accuracy: 0.8826 - val_loss: 0.3821 - val_categorical_accuracy: 0.9138


In [26]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

9/9 [==============================] - 3s 369ms/step - loss: 0.3588 - categorical_accuracy: 0.9132
[test loss, test accuracy]: [0.35881656408309937, 0.9131944179534912]
